In [5]:
import socket
import torch
from transformers import BertForSequenceClassification
import pickle

# Set up the client to connect to the server
def start_client(server_host='172.16.19.7', server_port=1700):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((server_host, server_port))
    print(f"Connected to server at {server_host}:{server_port}")

    return client_socket

# Load model
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

# Manually split the model: Second part of the model (classifier head)
class BertPart2(torch.nn.Module):
    def __init__(self, model):
        super(BertPart2, self).__init__()
        self.classifier = model.classifier  # Extract the classifier part of the model

    def forward(self, hidden_states):
        return self.classifier(hidden_states)  # Pass the hidden states through the classifier

# Create the second part of the model
model_part2 = BertPart2(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part2.to(device)

# Start the client and connect to the server
client_socket = start_client(server_host='172.16.19.7', server_port=1700)  # Change to your server's IP if needed

# Continuously receive batches from the server and process
while True:
    try:
        # Receive data from the server
        received_data = b''
        while True:
            chunk = client_socket.recv(4096)
            if not chunk:
                break
            received_data += chunk

        if not received_data:
            break  # Break if no more data is received

        # Deserialize the received data
        hidden_states = pickle.loads(received_data)

        # Convert the received data to a torch tensor and move to device
        hidden_states = torch.tensor(hidden_states).to(device)

        # Forward pass through the second part of the model
        outputs = model_part2(hidden_states)

        # Perform any additional steps, like loss computation or inference
        print(f"Processed batch output: {outputs}")

    except Exception as e:
        print(f"Error during processing: {e}")
        break

# Close the connection when done
client_socket.close()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Connected to server at 172.16.19.7:1700
Processed batch output: tensor([[[0.1730, 0.1164],
         [0.1552, 0.1834],
         [0.1654, 0.0787],
         ...,
         [0.2110, 0.0769],
         [0.2143, 0.1423],
         [0.2415, 0.1590]],

        [[0.1667, 0.1133],
         [0.1629, 0.1646],
         [0.1853, 0.0999],
         ...,
         [0.1970, 0.0999],
         [0.2118, 0.1301],
         [0.2541, 0.1112]],

        [[0.1493, 0.1432],
         [0.1572, 0.1869],
         [0.1605, 0.1101],
         ...,
         [0.2060, 0.0952],
         [0.2615, 0.1272],
         [0.2759, 0.1140]],

        ...,

        [[0.1816, 0.1370],
         [0.1469, 0.1482],
         [0.1722, 0.1022],
         ...,
         [0.2328, 0.0584],
         [0.2274, 0.0941],
         [0.2760, 0.1180]],

        [[0.1471, 0.1492],
         [0.1530, 0.1503],
         [0.1726, 0.1045],
         ...,
         [0.1844, 0.1206],
         [0.1975, 0.1624],
         [0.2444, 0.1139]],

        [[0.1784, 0.1285],
     

In [7]:
outputs[0]

tensor([[ 0.1730,  0.1164],
        [ 0.1552,  0.1834],
        [ 0.1654,  0.0787],
        [ 0.1800, -0.0280],
        [ 0.1743, -0.0449],
        [ 0.1943, -0.0607],
        [ 0.1695, -0.0091],
        [ 0.1854,  0.0215],
        [ 0.2449,  0.0053],
        [ 0.2659, -0.0124],
        [ 0.2998, -0.0553],
        [ 0.3253,  0.0160],
        [ 0.3075,  0.0604],
        [ 0.2925,  0.1245],
        [ 0.2925,  0.1520],
        [ 0.2817,  0.1395],
        [ 0.2883,  0.0852],
        [ 0.2660,  0.0466],
        [ 0.2556,  0.0212],
        [ 0.2487,  0.0496],
        [ 0.2228,  0.1331],
        [ 0.1691,  0.1274],
        [ 0.1829,  0.1120],
        [ 0.2244,  0.1050],
        [ 0.3061,  0.1180],
        [ 0.3351,  0.0746],
        [ 0.3608,  0.0763],
        [ 0.3816,  0.0860],
        [ 0.3670,  0.1395],
        [ 0.3309,  0.2108],
        [ 0.3272,  0.2167],
        [ 0.3063,  0.1720],
        [ 0.2855,  0.1556],
        [ 0.2654,  0.0438],
        [ 0.2440,  0.0228],
        [ 0.2299,  0

In [8]:
outputs[1]

tensor([[ 0.1667,  0.1133],
        [ 0.1629,  0.1646],
        [ 0.1853,  0.0999],
        [ 0.1903,  0.0071],
        [ 0.1867, -0.0271],
        [ 0.1871, -0.0622],
        [ 0.1952,  0.0053],
        [ 0.1934,  0.0142],
        [ 0.2327,  0.0144],
        [ 0.2795, -0.0446],
        [ 0.3267, -0.0318],
        [ 0.3379,  0.0184],
        [ 0.3160,  0.0715],
        [ 0.3222,  0.1226],
        [ 0.2836,  0.1478],
        [ 0.2525,  0.1267],
        [ 0.2680,  0.1042],
        [ 0.2875,  0.0795],
        [ 0.2761,  0.0395],
        [ 0.2567,  0.0896],
        [ 0.2302,  0.1378],
        [ 0.1934,  0.1232],
        [ 0.1918,  0.1362],
        [ 0.2717,  0.1244],
        [ 0.2945,  0.1343],
        [ 0.3321,  0.0857],
        [ 0.3889,  0.0531],
        [ 0.3855,  0.0477],
        [ 0.3849,  0.1393],
        [ 0.3488,  0.1975],
        [ 0.3079,  0.2048],
        [ 0.2986,  0.1967],
        [ 0.2780,  0.1252],
        [ 0.2592,  0.0931],
        [ 0.2363,  0.0373],
        [ 0.2326,  0

In [9]:
len(outputs)

16

In [10]:
model_part2

BertPart2(
  (classifier): Linear(in_features=128, out_features=2, bias=True)
)

In [11]:
import socket
import torch
from transformers import BertForSequenceClassification
import pickle

# Set up the client to connect to the server
def start_client(server_host='172.16.19.7', server_port=1700):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((server_host, server_port))
    print(f"Connected to server at {server_host}:{server_port}")

    return client_socket

# Load model
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)

# Manually split the model: Second half (classifier head)
class BertPart2(torch.nn.Module):
    def __init__(self, model):
        super(BertPart2, self).__init__()
        self.classifier = model.classifier  # Extract the classifier part of the model

    def forward(self, hidden_states):
        return self.classifier(hidden_states)  # Pass the hidden states through the classifier

# Create the second part of the model
model_part2 = BertPart2(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part2.to(device)

# Start the client and connect to the server
client_socket = start_client(server_host='172.16.19.7', server_port=1700)  # Change to your server's IP if needed

# Continuously receive batches from the server and process
while True:
    try:
        # Receive data from the server
        received_data = b''
        while True:
            chunk = client_socket.recv(4096)
            if not chunk:
                break
            received_data += chunk

        if not received_data:
            break  # Break if no more data is received

        # Deserialize the received data
        hidden_states = pickle.loads(received_data)

        # Convert the received data to a torch tensor and move to device
        hidden_states = torch.tensor(hidden_states).to(device)

        # Forward pass through the second part of the model
        outputs = model_part2(hidden_states)

        # Perform any additional steps, like loss computation or inference
        print(f"Processed batch output: {outputs}")

    except Exception as e:
        print(f"Error during processing: {e}")
        break

# Close the connection when done
client_socket.close()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Connected to server at 172.16.19.7:1700
Processed batch output: tensor([[[0.0801, 0.0803],
         [0.0544, 0.0860],
         [0.0579, 0.0831],
         ...,
         [0.0718, 0.0796],
         [0.0763, 0.0774],
         [0.0784, 0.0776]],

        [[0.0794, 0.0728],
         [0.0489, 0.0778],
         [0.0517, 0.0765],
         ...,
         [0.0742, 0.0698],
         [0.0789, 0.0696],
         [0.0813, 0.0693]],

        [[0.0620, 0.0875],
         [0.0367, 0.0965],
         [0.0408, 0.0900],
         ...,
         [0.0572, 0.0861],
         [0.0617, 0.0840],
         [0.0651, 0.0838]],

        ...,

        [[0.0643, 0.0860],
         [0.0432, 0.0895],
         [0.0448, 0.0849],
         ...,
         [0.0592, 0.0837],
         [0.0642, 0.0852],
         [0.0684, 0.0826]],

        [[0.0695, 0.0731],
         [0.0478, 0.0839],
         [0.0507, 0.0798],
         ...,
         [0.0683, 0.0706],
         [0.0716, 0.0692],
         [0.0740, 0.0714]],

        [[0.0711, 0.0746],
     

In [12]:
model_part2

BertPart2(
  (classifier): Linear(in_features=312, out_features=2, bias=True)
)

In [13]:
import socket
import torch
from transformers import BertForSequenceClassification
import pickle

# Set up the client to connect to the server
def start_client(server_host='127.0.0.1', server_port=1700):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((server_host, server_port))
    print(f"Connected to server at {server_host}:{server_port}")

    return client_socket

# Load model
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)

# Manually split the model: Classifier head
class BertPart2(torch.nn.Module):
    def __init__(self, model):
        super(BertPart2, self).__init__()
        self.classifier = model.classifier  # Extract the classifier part of the model

    def forward(self, hidden_states, pooled_output):
        # You can perform operations on hidden_states and pooled_output here.
        return self.classifier(hidden_states), self.classifier(pooled_output)

# Create the second part of the model
model_part2 = BertPart2(model)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_part2.to(device)

# Start the client and connect to the server
client_socket = start_client(server_host='172.16.19.7', server_port=1700)  # Change to your server's IP if needed

# Continuously receive batches from the server and process
while True:
    try:
        # Receive data from the server
        received_data = b''
        while True:
            chunk = client_socket.recv(4096)
            if not chunk:
                break
            received_data += chunk

        if not received_data:
            break  # Break if no more data is received

        # Deserialize the received data
        data = pickle.loads(received_data)
        hidden_states = torch.tensor(data['hidden_states']).to(device)
        pooled_output = torch.tensor(data['pooled_output']).to(device)

        # Forward pass through the second part of the model
        outputs_hidden, outputs_pooled = model_part2(hidden_states, pooled_output)

        # Perform any additional steps, like loss computation or inference
        print(f"Processed hidden states output: {outputs_hidden}")
        print(f"Processed pooled output: {outputs_pooled}")

    except Exception as e:
        print(f"Error during processing: {e}")
        break

# Close the connection when done
client_socket.close()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Connected to server at 172.16.19.7:1700
Processed hidden states output: tensor([[[ 1.8846e-02, -8.0754e-02],
         [-9.3658e-02, -1.8274e-02],
         [-1.5725e-01,  5.3438e-02],
         [-1.8658e-01, -1.1773e-01],
         [-1.6192e-01,  6.2647e-02],
         [-7.8981e-02,  2.1435e-02],
         [-1.2968e-01, -9.1606e-02],
         [ 4.2567e-02, -1.3776e-01],
         [ 4.9324e-02, -1.5062e-01],
         [-6.5254e-02, -1.1223e-01],
         [-8.0334e-02, -1.1276e-01],
         [-8.6222e-02, -1.2212e-01],
         [-8.3808e-02, -1.0657e-01],
         [-1.1660e-01, -1.2760e-01],
         [-1.0506e-01, -1.2915e-01],
         [-9.4517e-02, -1.0409e-01],
         [-1.2462e-01, -1.3303e-01],
         [-1.1084e-01, -1.2664e-01],
         [-9.7736e-02, -1.1102e-01],
         [-8.6261e-02, -1.0912e-01],
         [-5.7706e-02, -8.8994e-02],
         [-6.1600e-02, -9.3611e-02],
         [-6.2981e-02, -1.1349e-01],
         [-6.0895e-02, -1.0948e-01],
         [-8.7108e-02, -1.4991e-01],
   

In [15]:
pooled_output

tensor([[-1.4834e-02, -5.3914e-02, -1.8894e-02,  1.9982e-02, -1.3638e-02,
          1.2368e-02, -5.0215e-02,  1.9935e-02, -1.0283e-02, -5.8028e-02,
         -3.5619e-02,  5.6508e-03,  2.0197e-02, -4.6270e-02,  2.5706e-02,
          7.7902e-03, -1.4050e-02,  1.0448e-02,  3.9982e-03,  1.1972e-02,
          1.6616e-02, -2.0609e-03,  3.5185e-02, -2.0689e-02, -2.4612e-02,
         -3.1404e-02, -2.0241e-03,  3.8373e-02,  2.8636e-03, -2.1974e-02,
          1.3790e-03, -3.2166e-02, -4.3881e-02, -6.6052e-02, -4.0423e-04,
         -2.4887e-02, -2.8885e-02, -6.6117e-02, -2.7433e-02, -1.8742e-02,
         -4.6978e-03, -2.0703e-02, -4.8427e-03,  7.6686e-02,  1.1949e-02,
         -3.2272e-02,  9.4810e-03, -2.5523e-02, -1.9180e-02,  2.0887e-03,
         -9.4648e-02, -4.3184e-02,  2.1908e-02, -1.6801e-02, -1.5450e-02,
         -1.5052e-02, -4.8178e-02, -8.6782e-03,  3.8474e-04, -2.3081e-02,
          7.3196e-03, -2.8303e-02,  1.4755e-02,  8.6737e-03, -3.7712e-02,
         -2.6995e-02, -7.4640e-02,  2.

In [ ]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset

# Custom Dataset for IMDb
class IMDBDataset(Dataset):
    def __init__(self, tokenizer, split):
        self.tokenizer = tokenizer
        self.data = load_dataset('imdb', split=split)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            item['text'],
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        label = 1 if item['label'] == 'pos' else 0
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Set up the client to connect to the server
def start_client(server_host='127.0.0.1', server_port=1700):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((server_host, server_port))
    print(f"Connected to server at {server_host}:{server_port}")

    return client_socket

# Load model
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)

# Use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Start the client and connect to the server
client_socket = start_client(server_host='172.16.19.7', server_port=1700)  # Change to your server's IP if needed

# Create a DataLoader for the IMDb test dataset
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
test_dataset = IMDBDataset(tokenizer, split='test')
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

predictions = []
labels = []

# Continuously receive batches from the server and process
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    true_labels = batch['label'].cpu().numpy()  # Collect true labels for accuracy calculation
    labels.extend(true_labels)

    try:
        # Receive data from the server
        received_data = b''
        while True:
            chunk = client_socket.recv(4096)
            if not chunk:
                break
            received_data += chunk

        if not received_data:
            break  # Break if no more data is received

        # Deserialize the received data
        hidden_states = pickle.loads(received_data)

        # Convert the received data to a torch tensor and move to device
        hidden_states = torch.tensor(hidden_states).to(device)

        # Forward pass through the pooling and classifier head
        outputs = model(hidden_states)

        # Get predictions
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())

    except Exception as e:
        print(f"Error during processing: {e}")
        break

# Calculate and print accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the connection when done
client_socket.close()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Connected to server at 172.16.19.7:1700
